In [ ]:
import bw2data as bd
from collections import defaultdict
from tqdm import tqdm
from thefuzz import fuzz
from gsa_framework.utils import write_pickle, read_pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import lognorm
import copy
from scipy.stats import dirichlet
import numpy as np
import matplotlib.backends.backend_pdf
from pathlib import Path

import sys
sys.path.append("/Users/akim/PycharmProjects/akula")

from akula.implicit_markets import get_dirichlet_scales

In [ ]:
bd.projects.set_current('GSA for archetypes')
bd.databases

The inspiration for these virtual markets was the input of 'soybean' to 'market for soybean, feed' which has a reference product 'soybean, feed'. We can't just test exact matching, need to be a bit [more flexible](https://github.com/seatgeek/thefuzz) on these virtual markets.

In [ ]:
def similar(a, b):
    return fuzz.partial_ratio(a, b) > 90 or fuzz.ratio(a, b) > 40

In [ ]:
def find_uncertain_implicit_markets(database):
    db = bd.Database(database)

    found = {}
    
    for act in tqdm(db):
        rp = act.get("reference product")
        if not rp:
            continue
            
        inpts = defaultdict(list)
        for exc in act.technosphere():
            if exc.input == exc.output:
                continue
            elif exc['uncertainty type'] < 2:
                continue
            inpts[exc.input['reference product']].append(exc)
            
        for key, lst in inpts.items():
            if len(lst) > 1 and similar(rp, key) and 0.98 <= sum([exc['amount'] for exc in lst]) <= 1.02:
                found[act] = lst
            
    return found

def find_markets(database):
    db = bd.Database(database)

    found = {}
    
    for act in tqdm(db):
        if 'market group' in act['name']:
            continue
        if 'market' in act['name']:
            rp = act.get("reference product")
            if not rp:
                continue

            inpts = defaultdict(list)
            for exc in act.technosphere():
                if exc.input == exc.output:
                    continue
                inpts[exc.input['reference product']].append(exc)

            for key, lst in inpts.items():
                if len(lst) > 1 and rp==key and 0.98 <= sum([exc['amount'] for exc in lst]) <= 1.02:
                    found[act] = lst
            
    return found

In [ ]:
# ei_name = "ecoinvent 3.8 cutoff"
# found = find_uncertain_implicit_markets(ei_name)
# markets = find_markets(ei_name)
# write_pickle(found, "implicit_markets.pickle")
# write_pickle(markets, "normal_markets.pickle")
found = read_pickle("implicit_markets.pickle")
markets = read_pickle("normal_markets.pickle")

In [ ]:
ng = list(found)[32]
# ng, found[ng]

We can use the [dirichlet](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.dirichlet.html) to model parameters with a fixed sum, but this distribution is sensitive to the concentration values.

In [ ]:
get_beta_variance = lambda a,b: a*b/(a+b)**2/(a+b+1)
get_beta_skewness = lambda a,b: 2*(b-a)*((a+b+1)**0.5) / (a+b+2) / (a*b)**0.5

get_lognormal_variance = lambda loc, scale: (np.exp(scale**2)-1) * np.exp(2*loc+scale**2)
get_lognormal_skewness = lambda loc, scale: (np.exp(scale**2)+2) * ((np.exp(scale**2)-1)**0.5)

def get_dirichlet_scaling_factor(alpha_exc_dict):
    alphas = list(alpha_exc_dict.keys())
    beta = sum(alphas)
    alpha_threshold = np.mean(alphas)
    scaling_factors, scaling_factors_skewness = [], []
    for ialpha, iexc in alpha_exc_dict.items():
        if ialpha >= alpha_threshold:
            assert iexc['uncertainty type'] == 2
            loc = iexc['loc']
            scale = iexc['scale']
            beta_variance = get_beta_variance(ialpha, beta)
            lognormal_variance = get_lognormal_variance(loc, scale)
            beta_skewness = get_beta_skewness(ialpha, beta)
            lognormal_skewness = get_lognormal_skewness(loc, scale)
            scaling_factors.append(beta_variance / lognormal_variance * 2)
            scaling_factors_skewness.append(beta_skewness / lognormal_skewness)
    scaling_factor = np.mean(scaling_factors)
    scaling_factor_skewness = np.mean(scaling_factors_skewness)
#     print(scaling_factors, scaling_factor)
#     print(scaling_factors_skewness, scaling_factor_skewness)
    return max(scaling_factor, 50)

In [ ]:
# pdf = matplotlib.backends.backend_pdf.PdfPages("implicit_market_figures.pdf")
write_figs = Path("implicit_markets")
write_figs.mkdir(exist_ok=True, parents=True)

dist_ = {}
num_bins = 100
count=0

for ng, current in found.items():

    rows=len(current)

    showlegend = True
    x = np.array([exc['amount'] for exc in current])
    alpha = x.copy()
    alpha_exc_dict = {alpha[i]: current[i] for i in range(len(alpha))}
    scaling_factors = [get_dirichlet_scaling_factor(alpha_exc_dict), 250, 500]

    scaling_factors_str = [f"SF={sf:5.3f}" for sf in scaling_factors]
    fig = make_subplots(
        rows=rows, 
        cols=3,
        horizontal_spacing=0.2,
        subplot_titles=scaling_factors_str
    )

    for j,scaling_factor in enumerate(scaling_factors):
        rvs = dirichlet.rvs(alpha*scaling_factor, size=1000)
        for i,exc in enumerate(current):
            Y = rvs[:,i]
            bins_ = np.linspace(min(Y), max(Y), num_bins+1, endpoint=True)
            Y_samples, _ = np.histogram(Y, bins=bins_, density=True)
            # Given distribution
            assert exc['uncertainty type']==2
            loc = exc['loc']
            scale = exc['scale']  
            midbins = (bins_[1:]+bins_[:-1])/2
            Y_distr = lognorm.pdf(midbins, s=scale, scale=np.exp(loc))
            distance = np.sqrt(sum(Y_distr-Y_samples)**2)/max(Y_distr)

            fig.add_trace(
                go.Scatter(
                    x = midbins,
                    y = Y_samples,
                    line_color = 'blue',
                    name='Dirichlet samples',
                    showlegend=showlegend,
                ),
                row=i+1,
                col=j+1,
            )
            fig.add_trace(
                go.Scatter(
                    x = midbins,
                    y = Y_distr,
                    line_color = 'red',
                    name='Defined distribution',
                    showlegend=showlegend,
                ),
                row=i+1,
                col=j+1,
            )
            showlegend=False
            fig.update_yaxes(
                title_text=f"ED={distance:5.3f}",
                row=i+1,
                col=j+1,
            )
    fig.update_layout(
        width=700,
        height=250*rows,
        legend=dict(
            yanchor="top",
            y=-0.2,
            xanchor="left",
            x=0.01,
            orientation='h',
        )
    )
    fig.write_html(write_figs / "{}_{}_{}.html".format(count, ng['name'][:20], ng['location'][:3]))
    count += 1

In [ ]:
sb.displot(rvs[:, 1])

In [ ]:
rvs = dirichlet.rvs(alpha * 500, size=1000)

In [ ]:
sb.displot(rvs[:, 0])

In [ ]:
sb.displot(rvs[:, 1])

We can use these new values in Monte Carlo assessment (in place of the independent sampling which results in broken mass balances). The exact approach here will probably be different; for example, one could use trade statistics to create regional markets with much higher precision.

The underlying concepts in the following are documented in [bw_processing](https://github.com/brightway-lca/bw_processing) and [matrix_utils](https://github.com/brightway-lca/matrix_utils). In this notebook we will use in-memory datapackages for our fixes.

In [ ]:
import bw_processing as bwp

In [ ]:
indices_array = np.array([(exc.input.id, exc.output.id) for exc in found[ng]], dtype=bwp.INDICES_DTYPE)

# Redefine alpha to make sure order is consistent
# Transpose to get rows or exchange indices, columns of possible values
data_array = dirichlet.rvs(np.array([exc['amount'] for exc in found[ng]]) * 500, size=1000).T

# technosphere inputs must be flipped
flip_array = np.ones(len(found[ng]), dtype=bool)

In [ ]:
dp = bwp.create_datapackage()

In [ ]:
dp.add_persistent_array(
    matrix="technosphere_matrix",
    data_array=data_array,
    name="ng-fix-dz-es",
    indices_array=indices_array,
    flip_array=flip_array,
)

Compare Monte Carlo results with and without the fix

In [ ]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [ ]:
_, data_objs, _ = bd.prepare_lca_inputs({ng: 1}, method=ipcc)

Default is to use three datapackages: biosphere database, ecoinvent database, and LCIA method

In [ ]:
data_objs

In [ ]:
import bw2calc as bc

In [ ]:
lca = bc.LCA({ng.id: 1}, data_objs=data_objs, use_distributions=True)
lca.lci()
lca.lcia()

In [ ]:
unmodified = np.array([lca.score for _ in zip(lca, range(250))])

In [ ]:
fixed = bc.LCA({ng.id: 1}, data_objs=data_objs + [dp], use_arrays=True, use_distributions=True)
fixed.lci()
fixed.lcia()

In [ ]:
modified = np.array([fixed.score for _ in zip(fixed, range(250))])

Uncertainty for this example is not huge, so difference is not obvious

In [ ]:
np.mean(modified), np.std(modified), np.mean(unmodified), np.std(modified)

In [ ]:
for exc in found[ng]:
    lca.redo_lcia({exc.input.id: 1})
    print(lca.score)

In [ ]:
for exc in found[ng]:
    print(exc['scale'])

In [ ]:
sum([
    lca.technosphere_matrix[lca.dicts.product[row], lca.dicts.activity[col]]
    for row, col in indices_array
])

In [ ]:
sum([
    fixed.technosphere_matrix[fixed.dicts.product[row], fixed.dicts.activity[col]]
    for row, col in indices_array
])

In [ ]:
sb.displot(unmodified, kde=True)

In [ ]:
sb.displot(modified, kde=True)

# Check implementation

In [ ]:
import bw2data as bd
import bw_processing as bwp
from collections import defaultdict
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fs.zipfs import ZipFS
from scipy.stats import lognorm

import sys
sys.path.append("/Users/akim/PycharmProjects/akula")
from akula.implicit_markets import DATA_DIR

bd.projects.set_current('GSA for archetypes')

fp_implicit_markets = DATA_DIR / "implicit-markets.zip"
dp = bwp.load_datapackage(ZipFS(fp_implicit_markets))

In [ ]:
indices = dp.get_resource('implicit markets.indices')[0]
data = dp.get_resource('implicit markets.data')[0]
# distributions = dp.get_resource('implicit markets.distributions')[0]

In [ ]:
found = defaultdict(list)
data_act = defaultdict(list)
for col in list(set(indices['col'])):
    rows = indices[indices['col']==col]['row']
    act = bd.get_activity(int(col))
    for exc in act.exchanges():
        if exc.input.id in rows:
            found[act].append(exc)
            row_col = np.array((exc.input.id, col), dtype=indices.dtype)
            where = np.where(indices == row_col)[0][0]
            data_act[act].append(data[where])

In [ ]:
# pdf = matplotlib.backends.backend_pdf.PdfPages("implicit_market_figures.pdf")
write_figs = Path("implicit_markets")
write_figs.mkdir(exist_ok=True, parents=True)

dist_ = {}
num_bins = 100
count = 0
  
for ng, current in found.items():

    rows=len(current)
    showlegend = True
#     x = np.array([exc['amount'] for exc in current])
#     alpha = x.copy()
#     alpha_exc_dict = {alpha[i]: current[i] for i in range(len(alpha))}
#     scaling_factors = [get_dirichlet_scaling_factor(alpha_exc_dict), 250, 500]

#     scaling_factors_str = [f"SF={sf:5.3f}" for sf in scaling_factors]
    fig = make_subplots(
        rows=rows, 
        cols=1,
    )

    for i,exc in enumerate(current):
        Y = data_act[ng][i]
        bins_ = np.linspace(min(Y), max(Y), num_bins+1, endpoint=True)
        Y_samples, _ = np.histogram(Y, bins=bins_, density=True)
        # Given distribution
        assert exc['uncertainty type']==2
        loc = exc['loc']
        scale = exc['scale']  
        midbins = (bins_[1:]+bins_[:-1])/2
        Y_distr = lognorm.pdf(midbins, s=scale, scale=np.exp(loc))
        distance = np.sqrt(sum(Y_distr-Y_samples)**2)/max(Y_distr)

        fig.add_trace(
            go.Scatter(
                x = midbins,
                y = Y_samples,
                line_color = 'blue',
                name='Dirichlet samples',
                showlegend=showlegend,
            ),
            row=i+1,
            col=1,
        )
        fig.add_trace(
            go.Scatter(
                x = midbins,
                y = Y_distr,
                line_color = 'red',
                name='Defined distribution',
                showlegend=showlegend,
            ),
            row=i+1,
            col=1,
        )
        showlegend=False
        fig.update_yaxes(
            title_text=f"ED={distance:5.3f}",
            row=i+1,
            col=1,
        )
    fig.update_layout(
        width=300,
        height=250*rows,
        legend=dict(
            yanchor="top",
            y=-0.2,
            xanchor="left",
            x=0.01,
            orientation='h',
        )
    )
    fig.write_html(write_figs / "{}_{}_{}.html".format(count, ng['name'][:20], ng['location'][:3]))
    count += 1

# Plot implicit markets for paper 3

In [ ]:
names1 = [
    r'$\text{heat and power,}$',
    r'$\text{electricity production,}$',
    r'$\text{electricity production,}$',
    r'$\text{heat and power,}$',
    r'$\text{electricity production,}$',
]
names2 = [
    r'$\text{hard coal, RU}$',
    r'$\text{lignite, BA}$',
    r'$\text{hard coal, ME}$',
    r'$\text{lignite, RU}$',
    r'$\text{lignite, ME}$',
]

In [ ]:
color_gray_hex = "#b2bcc0"
color_darkgray_hex = "#485063"
color_black_hex = "#212931"
color_pink_rgb = "rgb(148, 52, 110)"
color_blue_rgb = "rgb(29,105,150)"
color_orange_rgb = "rgb(217,95,2)"
color_red_hex = "#ff2c54"
opacity=0.6
num_bins = 60

cols = 5
fig = make_subplots(
    rows=1, 
    cols=cols,
    horizontal_spacing=0.05,
    shared_yaxes=False,
#     subplot_titles=names2,
)
showlegend = True
for i,exc in enumerate(current):
    Y = data_act[ng][i]
    bins_ = np.linspace(min(Y), max(Y), num_bins+1, endpoint=True)
    Y_samples, _ = np.histogram(Y, bins=bins_, density=True)
    # Given distribution
    assert exc['uncertainty type']==2
    loc = exc['loc']
    scale = exc['scale']  
    midbins = (bins_[1:]+bins_[:-1])/2
    Y_distr = lognorm.pdf(midbins, s=scale, scale=np.exp(loc))
    distance = np.sqrt(sum(Y_distr-Y_samples)**2)/max(Y_distr)

    fig.add_trace(
        go.Scatter(
            x = midbins,
            y = Y_samples,
            line_color = color_darkgray_hex,
            name=r"$\text{Dirichlet samples}$",
            showlegend=showlegend,
            opacity=opacity,
            line=dict(color=color_blue_rgb, width=1, shape="hvh"),
            fill="tozeroy",
        ),
        row=1,
        col=i+1,
    )
    fig.add_trace(
        go.Scatter(
            x = midbins,
            y = Y_distr,
            line_color = color_red_hex,
            name=r"$\text{Defined lognormal distributions}$",
            showlegend=showlegend,
        ),
        row=1,
        col=i+1,
    )
    showlegend=False
    
fig.update_xaxes(
    title_text=r"$\text{Production volume share}$",
    showgrid=True,
    gridwidth=1,
    gridcolor=color_gray_hex,
    zeroline=True,
    zerolinewidth=1,
    zerolinecolor=color_black_hex,
    showline=True,
    linewidth=1,
    linecolor=color_gray_hex,
)

fig.update_yaxes(title_text=r"$\text{Frequency}$", col=1)
fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor=color_gray_hex,
    zeroline=True,
    zerolinewidth=1,
    zerolinecolor=color_black_hex,
    showline=True,
    linewidth=1,
    linecolor=color_gray_hex,
)

xpos = [0.08, 0.29, 0.5, 0.71, 0.92]
for i in range(cols):
    fig.add_annotation(
        {
            'font': {'size': 14},
            'showarrow': False,
            'text': names1[i],
            'x': xpos[i]-0.08,
            'xanchor': 'left',
            'xref': 'paper',
            'y': 1.2,
            'yanchor': 'bottom',
            'yref': 'paper'
        }
    )
    fig.add_annotation(
        {
            'font': {'size': 14},
            'showarrow': False,
            'text': names2[i],
            'x': xpos[i]-0.08,
            'xanchor': 'left',
            'xref': 'paper',
            'y': 1.05,
            'yanchor': 'bottom',
            'yref': 'paper'
        }
    )
    
fig.update_layout(
    width=220*cols,
    height=250,
    legend=dict(
        yanchor="middle",
        y=-0.7,
        xanchor="center",
        x=0.5,
        orientation='h',
        font=dict(size=13),
        bordercolor=color_darkgray_hex,
        borderwidth=1,
    ),
    margin=dict(t=40, b=10, l=10, r=0),
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
)
fig.write_image(Path("impicit_market_samples.eps"))

# fig.write_html(write_figs / "{}_{}_{}.html".format(count, ng['name'][:20], ng['location'][:3]))
# count += 1

# Plot regression figures for Dirichlet scales for paper 3

In [ ]:
from gsa_framework.utils import read_pickle

In [ ]:
# ei_name = "ecoinvent 3.8 cutoff"
# found = find_uncertain_implicit_markets(ei_name)
# markets = find_markets(ei_name)
# write_pickle(found, "implicit_markets.pickle")
# write_pickle(markets, "normal_markets.pickle")
ims = read_pickle("implicit_markets.pickle")
gms = read_pickle("generic_markets.pickle")

In [ ]:
LMEAN_ID, NEXC_ID = 0, 1

def get_market_features(markets):
    from bw2analyzer.econ import gini_coefficient
    X = []
    for i, act in enumerate(markets.keys()):
        exchanges = markets[act]
        amounts = np.array([exc['amount'] for exc in exchanges])
        mean = np.mean(amounts)
        lmean = np.mean(amounts[amounts>=mean])
        n_excs = len(exchanges)
        X.append([lmean, n_excs])
    X = np.array(X)
    X_ = 1/X[:,LMEAN_ID]**3
    return X_, X

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

xtrain = 1/Xtrain_base[:,curr_id]**3
fig.add_trace(
    go.Scatter(
        x=xtrain,
        y=ytrain,
        mode="markers",
#         marker = dict(
#             size=np.log(Xtrain_base[:,2])*10,  
#             showscale=True,
#             color=np.log(Xtrain_base[:,2])*10,
#         )
    ),
)

fig.update_layout(
    width=400,
    height=400,
)
# fig.update_xaxes(type="log")

xline = np.linspace(min(xtrain),max(xtrain),100)
fig.add_trace(
    go.Scatter(
        x=xline,
        y=reg.coef_[0]*xline,
        marker_color="red"
    )
)

# xtest = Xtest[:,curr_id]
# fig.add_trace(
#     go.Scatter(
#         x=xtest,
#         y=ytest,
#         marker_color="black"
#     )
# )
# fig = go.Figure()
# fig.add_trace(
#     go.Scatter3d(
#         x=Xtrain[:, MEAN_ID],
#         y=Xtrain[:, GINI_ID],
#         z=ytrain,
#         mode="markers"
#     )
# )
# fig.update_yaxes(range=(0,1000))
# fig.show()

In [ ]:
fig = go.Figure()

curr_id = 1

xtest = Xtest_base[:,curr_id]
fig.add_trace(
    go.Scatter(
        x=xtest,
        y=ytest,
        mode="markers",
        marker = dict(
            size=np.log(Xtest_base[:,2])*10,  
            showscale=True,
            color=np.log(Xtest_base[:,2])*10,
        )
    )
)

xline = np.linspace(min(xtest),max(xtest),100)
fig.add_trace(
    go.Scatter(
        x=xline,
        y=reg.coef_[0]/xline**3,
        marker_color="red"
    )
)

# [didn't work] Sampling with presamples

In [ ]:
ei_name = "ecoinvent 3.8 cutoff"

in_total = sum([exc['amount'] for exc in found[ng]])
out_total = 1
static_ratio = in_total / out_total if out_total != 0 else inf
static_balance = in_total - out_total

activity_params = []
for i,exc in enumerate(found[ng]):
#     if 'formula' in exc:
#         print(i)
#         break
    param_name = f"market_param_{i}"
    activity_params.append(
        {
            'name': param_name,
            'amount': exc.get('amount', 0),
            'uncertainty type': exc.get('uncertainty type', 0),
            'loc': exc.get('loc', exc.get('amount', 0)),
            'scale': exc.get('scale'),
            'negative': exc.get('negative', False),
            'database': ei_name,
            'code': ng.get('code'),
        }
    )
    if exc.get('uncertainty type', 0) > 1:
        exc['formula'] = "{} * scaling".format(param_name)
    else:
        exc['formula'] = param_name
    exc.save()
    if exc.get('variable name', False):
        exc['variable name temp'] = exc['variable name']
        exc['variable name'] = []
        exc.save()
    
activity_params.append(
    {
        'name': 'static_ratio',
        'database': ei_name,
        'code': ng['code'],
        'amount': static_ratio,
        'uncertainty type': 0,
        'loc': static_ratio,
    }
)
out_term = "1"
const_in_term = "0"
var_in_term = "(market_param_0 + market_param_1)"
activity_params.append(
    {
        'name': 'scaling',
        'formula': "({}*{}-{})/({})".format(static_ratio, out_term, const_in_term, var_in_term),
        'database': ei_name,
        'code': ng['code'],
    },
)
activity_params.append(
    {
        'name': 'ratio',
        'formula': "(scaling * {} + {})/{}".format(var_in_term, const_in_term, out_term),
        'database': ei_name,
        'code': ng['code'],
    },
)

group = 'my_market_2022_04'
iterations = 10
bd.parameters.new_activity_parameters(activity_params, group, True)
# bd.parameters.add_exchanges_to_group(group, ng)
bd.parameters.recalculate()
# pbm = PBM(group)
# pbm.load_parameter_data()
# pbm.calculate_stochastic(iterations, update_amounts=True)
# pbm.calculate_matrix_presamples()

In [ ]:
# Rescale lognormal distributions
iterations = 1000
Y_samples = np.zeros((0,iterations))
for i,exc in enumerate(found[ng]):
    # Given distribution
    if exc['uncertainty type']==2:
        loc = exc['loc']
        scale = exc['scale']  
#         x = np.linspace(
#             lognorm.ppf(0.01, s=scale, scale=np.exp(loc)), 
#             lognorm.ppf(0.99, s=scale, scale=np.exp(loc)), 
#             num_bins,
#         )
        x = np.linspace(0,1,iterations+1)
        Y_samples = np.vstack([Y_samples, lognorm.pdf(x[1:], s=scale, scale=np.exp(loc))])
Y_scaled = Y_samples / Y_samples.sum(axis=0)
Y_scaled = Y_scaled.T
Y_scaled.shape